In [ ]:
import os
import sc2reader

entries = os.scandir('C:/Users/Maguro/Documents/StarCraft II/Accounts/114803619/1-S2-1-4189373/Replays/Multiplayer')

main_player_name = 'Maguro'
amon_forces = ['Amon','Infested','Salamander','Void Shard','Hologram','Moebius', "Ji'nara" ]
duplicating_units = ['HotSRaptor']
skip_strings = ['placement', 'placeholder', 'dummy','cocoon']
rep_loaded = 0
rep_attempted = 0
revival_types = {'KerriganReviveCocoon':'K5Kerrigan', 'AlarakReviveBeacon':'AlarakCoop','ZagaraReviveCocoon':'ZagaraVoidCoop','DehakaCoopReviveCocoonFootPrint':'DehakaCoop','NovaReviveBeacon':'NovaCoop','ZeratulCoopReviveBeacon':'ZeratulCoop'}
possible_commanders = set()

def contains_skip_strings(pname):
    """ checks if any of skip strings is in the pname """
    lowered_name = pname.lower()
    for item in skip_strings:
        if item in lowered_name:
            return True
    return False

def check_amon_forces(alist, string):
    """ Checks if any word from the list is in the string """
    for item in alist:
        if item in string:
            return True
    return False

unit_type_dict_maguro = {}
#structure: {unitType : [#created, #died, #kills, #KD]}
unit_type_dict_amon = {}
#structure: {unitType : [#created, #died, #kills, #KD]}

for entry in entries:  
    if entry.is_file() and entry.name.endswith('.SC2Replay') and not('Test Map' in entry.name) and not('[MM]' in entry.name) and not('3P CoA' in entry.name):
        #try loading replay
        rep_attempted +=1
        try:
            replay = sc2reader.load_replay(entry.path,load_level=3)             
        except:
            print('//Failed to load replay',entry.name)
            continue
      
        is_coop_map = False
        player_commander = ''
        main_player = 1
        amon_players = []

        for per in replay.person:
            #check for commanders to exclude other replays
            try:
                if replay.person[per].commander != '':
                    if main_player_name in str(replay.person[per]): 
                        player_commander = replay.person[per].commander
            except:
                pass

            #find main player
            if main_player_name in str(replay.person[per]): 
                main_player = per

            #find Amon players
            if check_amon_forces(amon_forces,str(replay.person[per])) and per > 2:
                amon_players.append(per)

        if player_commander == '':
            continue

        rep_loaded += 1
        possible_commanders.add(player_commander)

        # print(player_commander)

        unit_dict = {}
        #structure: {unit_id : [UnitType, Owner]}

        #go through game events
        for event in replay.events:  
            if event.name == 'UnitBornEvent' or event.name == 'UnitInitEvent':
                #save into unit dict
                try:
                    unit_dict[str(event.unit_id)] = [event.unit_type_name,str(event.unit_controller)[7]]
                except:
                    #unit created without for no player
                    pass

                #certain hero units don't die, instead lets track their revival beacons/cocoons
                if event.unit_type_name in revival_types and main_player == event.control_pid and event.second > 0:
                    unit_type_dict_maguro[(revival_types[event.unit_type_name],player_commander)][1] += 1
                    unit_type_dict_maguro[(revival_types[event.unit_type_name],player_commander)][0] += 1

                #save stats for units created
                if main_player == event.control_pid:
                    unit_type = event.unit_type_name

                    if contains_skip_strings(unit_type):
                        continue

                    if (unit_type,player_commander) in unit_type_dict_maguro:
                        unit_type_dict_maguro[(unit_type,player_commander)][0] += 1
                    else:
                        unit_type_dict_maguro[(unit_type,player_commander)] = [1,0,0,0]

                if event.control_pid in amon_players:
                    unit_type = event.unit_type_name

                    if contains_skip_strings(unit_type):
                        continue

                    if unit_type in unit_type_dict_amon:
                        unit_type_dict_amon[unit_type][0] += 1
                    else:
                        unit_type_dict_amon[unit_type] = [1,0,0,0]


            if event.name == 'UnitDiedEvent':    

                try:
                    killing_unit_id = str(event.killing_unit_id)
                    killed_unit_type = unit_dict[str(event.unit_id)][0]
                    losing_player = int(unit_dict[str(event.unit_id)][1])

                    if killing_unit_id in unit_dict:
                        killing_unit_type = unit_dict[killing_unit_id][0]
                    else:
                        killing_unit_type = 'NoUnit'

                    if contains_skip_strings(killed_unit_type): #skip placeholders, dummies, cocoons
                        continue   
                
                    #update kills
                    if (killing_unit_id in unit_dict) and (killing_unit_id != str(event.unit_id)) and losing_player != event.killing_player_id:
                      
                        if main_player == event.killing_player_id:

                            if (killing_unit_type,player_commander) in unit_type_dict_maguro:
                                unit_type_dict_maguro[(killing_unit_type,player_commander)][2] += 1
                            else:
                                unit_type_dict_maguro[(killing_unit_type,player_commander)] = [0,0,1,0]

                        if event.killing_player_id in amon_players:

                            if killing_unit_type in unit_type_dict_amon:
                                unit_type_dict_amon[killing_unit_type][2] += 1
                            else:
                                unit_type_dict_amon[killing_unit_type] = [0,0,1,0]
                   
                    #update unit deaths
                    #fix for raptors that are counted each time they jump (as death and birth)
                    if main_player == losing_player and event.second > 0 and killed_unit_type in duplicating_units and killed_unit_type == killing_unit_type and losing_player == event.killing_player_id:
                        unit_type_dict_maguro[(killed_unit_type,player_commander)][0] -= 1
                        continue

                    if main_player == losing_player and event.second > 0: #don't count deaths on game init
                        unit_type_dict_maguro[(killed_unit_type,player_commander)][1] += 1

                    if losing_player in amon_players and event.second > 0:

                        unit_type_dict_amon[killed_unit_type][1] += 1           
                except:
                    pass
                    #some units aren't created for any player, and won't be saved into unit dict

# print(unit_type_dict_maguro)
print(f'---->Loaded {round(100*rep_loaded/rep_attempted,1)}% replays ({rep_loaded}/{rep_attempted}).')

In [ ]:
from UnitNameDict7 import UnitNameDict

def calculate_KD(pdit):
    """ Calculates K/D, """
    for item in pdit:
        if pdit[item][1] != 0:
            pdit[item][3] = pdit[item][2]/pdit[item][1]
        else:
            pdit[item][3] = float(pdit[item][2])  

                    
def save_dict(pdict,pname,sep):
    """ Saves dictionary into text file """
    temp_string = f'Unit type{sep}Commander{sep}created{sep}died{sep}kills{sep}K/D'
    for key in pdict:
        if len(key) == 2:
            temp_string = f'{temp_string}\n{key[0]}{sep}{key[1]}{sep}{pdict[key][0]}{sep}{pdict[key][1]}{sep}{pdict[key][2]}{sep}{pdict[key][3]}'
        else:
            temp_string = f'{temp_string}\n{key}{sep}{pdict[key][0]}{sep}{pdict[key][1]}{sep}{pdict[key][2]}{sep}{pdict[key][3]}'
            
    with open(pname, 'w') as file:       
        file.write(temp_string)


def switch_names(pdict):
    """ Changes names to that in unit dictionary, sums duplicates. This will mess up k/d calculation """
    temp_dict = {}
    for key in pdict:
        name = key[0]
        if key[0] in UnitNameDict:
            name = UnitNameDict[key[0]]

        if (name,key[1]) in temp_dict:
            for a in range(0,len(temp_dict[(name,key[1])])):
                temp_dict[(name,key[1])][a] += pdict[key][a]
        else:
            temp_dict[(name,key[1])] = pdict[key]

    return temp_dict

#MAGURO
sorted_maguro = {k:v for k,v in sorted(unit_type_dict_maguro.items(), reverse = True, key=lambda item: item[1][2])} #sorts by number of create (0), lost (1), kills (2), K/D (3)
save_dict(sorted_maguro,f'Log{main_player_name}RAW.csv',',')

sorted_maguro = switch_names(sorted_maguro)
calculate_KD(sorted_maguro)
save_dict(sorted_maguro,f'Log{main_player_name}.csv',',')

#AMON
sorted_amon = {k:v for k,v in sorted(unit_type_dict_amon.items(), reverse = True, key=lambda item: item[1][1])}
save_dict(sorted_amon,'LogAmonRAW.csv',',')

sorted_amon = switch_names(sorted_amon)
calculate_KD(sorted_amon)
save_dict(sorted_amon,'LogAmon.csv',',')

In [ ]:
### VISUALIZATION
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

units_to_remove = ['HHScrapPickup','BiomassPickup','SOAPylonPowerUnit','KerriganVoidCoopEconDropLT1','KerriganVoidCoopEconDrop1','AlarakSupplicantWarpTrainCreator','KerriganVoidCoopEconDrop2','KerriganVoidCoopEconDrop3','KerriganVoidCoopEconDrop4','KerriganVoidCoopEconDrop5','AlarakSupplicantWarpTrainDummy','EssencePickup','LocustMPPrecursor','Egg', 'Larva','HHMagneticMine_SpawnerUnit']

units_to_remove_for_created = ['PurificationOrbSmall','TychusMedicTransportUnit','FenixTalisAdeptPhaseShift','FenixAdeptShade','HHD8SingleCluster','HHD8ClusterBomb','HHD8CenterCluster','HHD8CenterClusterUpgrade','SOAThermalLanceTargeter','SoACasterArtanis','Broodling','Liberator_BlackOpsSpawnerUnit','Marine_BlackOpsSpawnerUnit','Raven_BlackOpsSpawnerUnit','Bio-Mechanical Repair Drone','Mecha Larva','ZeratulDisruptorPhased','Locust',"Brutalisk's Symbiote","Leviathan's Symbiote",'Scarab','Primal Locust','Explosive Creeper','Interceptor',"Reaper's KD8 Charge",'Drone','Probe','SCV','Toxic Nest','Overlord','Mecha Overlord','Pylon','Extractor','Refinery','Assimilator','Gateway','Warp Gate','Nexus','Hatchery','Command Center','Orbital Command','Warp Stargate','Primal Drone','Stargate','Supply Depot','Omega Worm','Creep Tumor','Automated Refinery','SpiderMine','Tech Lab','Reactor','SupplyDepotDrop','Mecha Broodling','Infested SCV','Factory','Orbital Assimilator','Forge','Evolution Chamber','Infestation Pit','Cybernetics Core','Primal Hive','Primal Warden','Fenix','NovaGriffinBombingRunStrafer','NovaGriffinBombingRunTargeter','NovaGriffinBombingRunDummyAltLMWeapon','NovaGriffinBombingRunDummyLMWeapon','Barracks','Tech Reactor','Spire','Ghost_BlackOpsSpawnerUnit','Spider Mine','Roach Warren','Purifier Conclave','Robotics Facility','Robotics Bay','Fleet Beacon','Twilight Council','SOAPylonPowerAllyUnit','Warhound Turret','Auto-Turret','SCV','Command Center',"Joeyray's Bar",'Refinery',"Engineering Bay",'Revitalizer','Medivac Platform','Omega Worm','Muscle Armory',"Gunslingers' Hideout","CODFlatbedTruck","Fixers' Safehouse",'CoopCasterTychus','Nydus Network','NovaGriffinTransportUnit','Mecha Drone','TrooperMengskWeaponImprovedPickup','TrooperMengskWeaponFlamethrowerPickup','TrooperMengskWeaponAAPickup','BunkerDepotMengskDrop','Thermal Lance','Infested Tech Lab','Solar Forge','Purifier Beam','Odin','Infested Missile Turret']


def get_df(pdict,player):
    DF = pd.DataFrame.from_dict(pdict, orient='index')
    DF.reset_index(inplace = True)
    DF.columns=['UnitType','Created','Lost','Kills','K/D']

    if player == True:
        DF['Commander'] = ''
        for a in range(0,len(DF.index)): 
            DF.iloc[a,5] = DF.iloc[a,0][1]
            DF.iloc[a,0] = DF.iloc[a,0][0]
    return DF

MaguroDF = get_df(sorted_maguro, True)
# print(MaguroDF)
bg_color = '#060606'
plt.rcParams['axes.facecolor'] = bg_color
plt.rcParams['figure.facecolor'] = bg_color
plt.rcParams['figure.dpi'] = 300


def pie_chart(pcommander, pparameter):
    TDF = pd.DataFrame(columns=['UnitType','Created','Lost','Kills','K/D','Commander'])
    TDF = MaguroDF[MaguroDF['Commander'] == pcommander]

    #remove some items
    for item in units_to_remove:
        TDF = TDF[TDF['UnitType'] != item]

    if pparameter == 'Created':
        for item in units_to_remove_for_created:
            TDF = TDF[TDF['UnitType'] != item]

    #remove those with too low percentage
    total_sum = TDF.sum(axis=0)[pparameter]
    threshold = 0.01
    if pcommander == 'Stukov'and pparameter == 'Created':
        threshold = 0.005

    TDF = TDF[TDF[pparameter] > total_sum*threshold]

    start_angle = 0
    if pparameter == 'Created':
        if pcommander == 'Dehaka':
            start_angle = 45
        elif pcommander == 'Nova':
            start_angle = 75
        elif pcommander == 'Stetmann':
            start_angle = 38
        elif pcommander == 'Abathur':
            start_angle = 30
        elif pcommander == 'Artanis':
            start_angle = 20
        elif pcommander == 'Vorazun':
            start_angle = 20
        elif pcommander == 'Zeratul':
            start_angle = -45
        elif pcommander == 'Karax':
            start_angle = 30
            
    if pparameter == 'Kills':
        if pcommander == 'Tychus':
            start_angle = 10
        if pcommander == 'Zeratul':
            start_angle = 10

    pie = plt.pie(TDF[pparameter], labels=TDF['UnitType'], autopct='%1.1f%%', startangle=start_angle, pctdistance=0.80,textprops={'color':"w"})

    #draw circle
    centre_circle = plt.Circle((0,0),0.60,fc=bg_color)
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    plt.title(pcommander, color = 'white', fontsize = 20)
    center_font = 18
    if pparameter == 'Kills':
        center_font = 24

    plt.suptitle(pparameter.upper(), color = '#444444', fontsize = center_font, horizontalalignment='center', verticalalignment='center', y=0.5,x=0.510, fontweight='bold')
    # plt.show()
    plt.savefig(f'./Charts/{pcommander}_{pparameter}.png', facecolor=fig.get_facecolor())
    plt.close('all') #needs to be closed, otherwise following charts will be messed up


print(possible_commanders)
for commanderiter in possible_commanders:
    pie_chart(commanderiter,'Kills')
    pie_chart(commanderiter,'Created')